In [ ]:
import pandas as pd

df = pd.read_csv('/content/combined_dataset.csv')
print(df.shape)

df.head()

(2522, 2)


,QUESTION,BT LEVEL
0,"Suppose prices of two goods are constant, expl...",Comprehension
1,Explain the concept of price leadership observ...,Comprehension
2,Define profit. Briefly explain how accounting ...,Comprehension
3,Describe the assumptions of monopolistic compe...,Comprehension
4,Explain the meaning of the law of diminishing ...,Comprehension


In [ ]:
df['BT LEVEL'].value_counts()

,count
BT LEVEL,
Comprehension,961
Knowledge,344
Evaluation,318
Application,316
Analysis,304
Synthesis,279


In [ ]:
import pandas as pd

min_samples = 279

# Convert BT LEVEL to uppercase to match the sampling criteria
df["BT LEVEL"] = df["BT LEVEL"].str.upper()

# Check the number of available samples in each category
bt_levels = ["COMPREHENSION", "KNOWLEDGE", "EVALUATION", "APPLICATION", "ANALYSIS", "SYNTHESIS"]

# Initialize empty DataFrames to store the sampled data
df_samples = []

for bt_level in bt_levels:
    df_bt_level = df[df["BT LEVEL"] == bt_level]
    available_samples = df_bt_level.shape[0]
    samples_to_draw = min(available_samples, min_samples)

    # Debugging: Print the number of available samples for each BT level
    print(f"{bt_level}: available samples = {available_samples}, samples to draw = {samples_to_draw}")

    if available_samples > 0:
        # Sample the data with replacement
        df_sample = df_bt_level.sample(samples_to_draw, random_state=2022, replace=True)
        df_samples.append(df_sample)
    else:
        print(f"No samples available for BT level: {bt_level}")

# Concatenate all sampled DataFrames
if df_samples:
    df_sampled = pd.concat(df_samples)
else:
    df_sampled = pd.DataFrame(columns=df.columns)

print(df_sampled)


COMPREHENSION: available samples = 961, samples to draw = 279
KNOWLEDGE: available samples = 344, samples to draw = 279
EVALUATION: available samples = 318, samples to draw = 279
APPLICATION: available samples = 316, samples to draw = 279
ANALYSIS: available samples = 304, samples to draw = 279
SYNTHESIS: available samples = 279, samples to draw = 279
                                               QUESTION       BT LEVEL
2266   Find the errors and rewrite the corrected codes.  COMPREHENSION
419   Explain FIVE (5) key components of organizatio...  COMPREHENSION
336   Sketch ONE (1) organic object in one layout to...  COMPREHENSION
1135  Describe the major functions or roles of soil ...  COMPREHENSION
1368  In your own words, how would you summarize the...  COMPREHENSION
...                                                 ...            ...
1975  Explain each inventory control method and sugg...      SYNTHESIS
1542                              Develop a hypothesis.      SYNTHESIS
1744  D

In [ ]:
df_sampled['BT LEVEL'].value_counts()

,count
BT LEVEL,
COMPREHENSION,279
KNOWLEDGE,279
EVALUATION,279
APPLICATION,279
ANALYSIS,279
SYNTHESIS,279


In [ ]:
df_balanced = df_sampled.copy()
df_balanced["BT LEVEL"].value_counts()

,count
BT LEVEL,
COMPREHENSION,279
KNOWLEDGE,279
EVALUATION,279
APPLICATION,279
ANALYSIS,279
SYNTHESIS,279


In [ ]:
df_balanced.head()

,QUESTION,BT LEVEL
2266,Find the errors and rewrite the corrected codes.,COMPREHENSION
419,Explain FIVE (5) key components of organizatio...,COMPREHENSION
336,Sketch ONE (1) organic object in one layout to...,COMPREHENSION
1135,Describe the major functions or roles of soil ...,COMPREHENSION
1368,"In your own words, how would you summarize the...",COMPREHENSION


In [ ]:
df_balanced['label_num'] = df_balanced['BT LEVEL'].map({
    "COMPREHENSION": 0,
    "KNOWLEDGE": 1,
    "EVALUATION": 2,
    "APPLICATION": 3,
    "ANALYSIS": 4,
    "SYNTHESIS": 5

})

#checking the results
df_balanced.head(5)

,QUESTION,BT LEVEL,label_num
2266,Find the errors and rewrite the corrected codes.,COMPREHENSION,0
419,Explain FIVE (5) key components of organizatio...,COMPREHENSION,0
336,Sketch ONE (1) organic object in one layout to...,COMPREHENSION,0
1135,Describe the major functions or roles of soil ...,COMPREHENSION,0
1368,"In your own words, how would you summarize the...",COMPREHENSION,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_balanced['QUESTION'],
    df_balanced['label_num'],
    test_size=0.2,
    random_state=2022,
    stratify=df_balanced.label_num
)

In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (1339,)
Shape of X_test:  (335,)


In [ ]:
X_train.head()

,QUESTION
1610,Compose a complete C program that reads text s...
871,Explain the possible flow patterns in the culv...
383,Describe material gathering process.
1542,Develop a hypothesis.
1851,Explain Greimas square tool and discuss how it...


In [ ]:
y_train.value_counts()

,count
label_num,
2,224
5,223
0,223
1,223
3,223
4,223


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 1))),        #using the ngram_range parameter
     ('Multi NB', MultinomialNB())
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.68      0.70        56
           1       0.87      0.71      0.78        56
           2       0.74      0.89      0.81        55
           3       0.72      0.77      0.74        56
           4       0.92      0.80      0.86        56
           5       0.80      0.88      0.84        56

    accuracy                           0.79       335
   macro avg       0.79      0.79      0.79       335
weighted avg       0.79      0.79      0.79       335



In [ ]:
#1. create a pipeline object
clf = Pipeline([
    ('vectorizer_1_2_gram', CountVectorizer(ngram_range = (1, 2))),        #using the ngram_range parameter
     ('Multi NB', MultinomialNB())
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.79      0.75        56
           1       0.91      0.73      0.81        56
           2       0.74      0.91      0.81        55
           3       0.79      0.82      0.81        56
           4       0.98      0.80      0.88        56
           5       0.84      0.84      0.84        56

    accuracy                           0.81       335
   macro avg       0.83      0.82      0.82       335
weighted avg       0.83      0.81      0.82       335



In [ ]:
#1. create a pipeline object
clf = Pipeline([
    ('vectorizer_1_3_grams', CountVectorizer(ngram_range = (1, 3))),        #using the ngram_range parameter
     ('Multi NB', MultinomialNB())
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.79      0.75        56
           1       0.91      0.71      0.80        56
           2       0.74      0.91      0.81        55
           3       0.79      0.82      0.81        56
           4       0.96      0.79      0.86        56
           5       0.82      0.84      0.83        56

    accuracy                           0.81       335
   macro avg       0.82      0.81      0.81       335
weighted avg       0.82      0.81      0.81       335



In [ ]:
### utlity function for pre-processing the text
import spacy

# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)

In [ ]:
df_balanced['preprocessed_question'] = df_balanced['QUESTION'].apply(preprocess)

In [ ]:
df_balanced.head()

,QUESTION,BT LEVEL,label_num,preprocessed_question
2266,Find the errors and rewrite the corrected codes.,COMPREHENSION,0,find error rewrite correct code
419,Explain FIVE (5) key components of organizatio...,COMPREHENSION,0,explain 5 key component organization crisis de...
336,Sketch ONE (1) organic object in one layout to...,COMPREHENSION,0,sketch 1 organic object layout harmony shape c...
1135,Describe the major functions or roles of soil ...,COMPREHENSION,0,describe major function role soil microbe bene...
1368,"In your own words, how would you summarize the...",COMPREHENSION,0,word summarize story


In [ ]:
df_balanced.QUESTION[419]

'Explain FIVE (5) key components of organization crisis definition.'

In [ ]:
df_balanced.preprocessed_question[419]

'explain 5 key component organization crisis definition'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_balanced.preprocessed_question,
    df_balanced.label_num,
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df_balanced.label_num
)

In [ ]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 1))),        #using the ngram_range parameter
     ('Multi NB', MultinomialNB())
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.66      0.64        56
           1       0.84      0.77      0.80        56
           2       0.81      0.84      0.82        55
           3       0.75      0.77      0.76        56
           4       0.89      0.84      0.86        56
           5       0.86      0.89      0.88        56

    accuracy                           0.79       335
   macro avg       0.80      0.79      0.79       335
weighted avg       0.80      0.79      0.79       335



In [ ]:
#1. create a pipeline object
clf = Pipeline([
    ('vectorizer_bow', CountVectorizer(ngram_range = (1, 2))),        #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))
print(X_test)
print(y_pred)

              precision    recall  f1-score   support

           0       0.66      0.71      0.68        56
           1       0.90      0.77      0.83        56
           2       0.77      0.85      0.81        55
           3       0.77      0.79      0.78        56
           4       0.92      0.80      0.86        56
           5       0.86      0.91      0.89        56

    accuracy                           0.81       335
   macro avg       0.81      0.81      0.81       335
weighted avg       0.81      0.81      0.81       335

2148    provide real life example conflict interest si...
672           Fermi Dirac function step function t = 0 k.
120                  Discuss 4 economic role money market
720     time JIT attempt remove delay add value explai...
1688                                        factor change
                              ...                        
158     distinguish stock dividend stock split provide...
363                                write 4 purpose 

In [ ]:
#1. create a pipeline object
clf = Pipeline([
    ('vectorizer_1_3_grams', CountVectorizer(ngram_range = (1, 3))),        #using the ngram_range parameter
     ('Multi NB', MultinomialNB())
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.71      0.68        56
           1       0.90      0.77      0.83        56
           2       0.77      0.85      0.81        55
           3       0.77      0.79      0.78        56
           4       0.92      0.80      0.86        56
           5       0.86      0.91      0.89        56

    accuracy                           0.81       335
   macro avg       0.81      0.81      0.81       335
weighted avg       0.81      0.81      0.81       335



In [ ]:
ques = ["what is the currency of USA ?"]
pre_quest = [preprocess(ques[0])]
clf.predict(pre_quest)

array([1])